In [2]:

from enum import Enum
from keras.preprocessing.sequence import pad_sequences
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,roc_curve,roc_auc_score,classification_report,f1_score,recall_score
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Flatten
from keras.layers import LSTM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylab
use_colab = True
if use_colab:
  from google.colab import drive


Using TensorFlow backend.


In [3]:
if use_colab:
  drive_name = '/content/drive'
  drive.mount(drive_name)
  drive_2541_A1_folder = 'CSC2541_A1'
  drive_location = drive_name + '/My Drive/' + drive_2541_A1_folder  # Change this to where your files are located
else:
  # set the drive_location variable to whereever the extracted contents are.
  drive_location = ''

data_location = drive_location + '/' + 'hypertension_patients.gz'
PARTIALLY_TRAINED_MODEL = drive_location + '/' + 'hypertension_charts.gz'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Read the file into pandas dataframe
hypertension_patients = pd.read_csv(data_location, compression='gzip')

In [0]:
# Read the file into pandas dataframe
hypertension_charts = pd.read_csv(PARTIALLY_TRAINED_MODEL,compression='gzip')

In [0]:
class measurement(Enum):
    HEART_RATE = 220045
    RESPIRATORY_RATE = 220210
    O2_SATURATION = 220277
    BLOOD_PRESSURE = 220181

In [7]:
for values in measurement:
    #filter for measurement value  values.value
    msr_val = hypertension_charts [hypertension_charts['itemid'] == values.value]

    #filtering out heart_rate values having count <2
    rm_ptnt = msr_val.groupby(['subject_id','hadm_id'],as_index = False).valuenum.agg(['count']).reset_index() 
    rm_ptnt = rm_ptnt.loc[rm_ptnt['count']<2]
    msr_val = msr_val.loc[~(msr_val['subject_id'].isin(rm_ptnt['subject_id']) & rm_ptnt['hadm_id'].isin(rm_ptnt['hadm_id']))]
    
    #filtering out top 500 values for each admission_id and subject_id
    msr_val = msr_val.groupby(['subject_id','hadm_id']).head(500).reset_index() 
    
#preparing data for LSTM
    
    #1. Ordering the records by charttime
    new_df = msr_val.sort_values(['hadm_id','subject_id','charttime'], ascending=[True, True,True])

    #groups by subject_id and hadm_id and makes a corresponding list of of all measurements values
    filtered_data_lstm = new_df.groupby(['subject_id','hadm_id']).valuenum.apply(list).reset_index()

    #2.merge one hospital_id admission to 
    filtered_data_lstm = pd.merge(hypertension_patients, filtered_data_lstm, on=['subject_id','hadm_id'], how='inner')

    filtered_data_lstm = filtered_data_lstm.drop(['subject_id','hadm_id'],axis=1)
    
    #Splitting training into test and train
    df_test,df_train = filtered_data_lstm.loc[filtered_data_lstm['train'] == 0],filtered_data_lstm.loc[filtered_data_lstm['train'] == 1]

    X_train,X_test = df_train.drop(['hypertension','train'],axis=1),df_test.drop(['hypertension','train'],axis=1)


    Y_train,Y_test = df_train[['hypertension']].values.squeeze(),df_test[['hypertension']].values.squeeze()
    
    #X_train_temp = X_train.values.tolist()
    X_train = pd.Series(X_train['valuenum'], index=X_train.index)
    X_train= np.array(pad_sequences(X_train))
    X_train = np.expand_dims(X_train, axis=2)
    
    X_test = pd.Series(X_test['valuenum'], index=X_test.index)
    X_test= np.array(pad_sequences(X_test))
    X_test = np.expand_dims(X_test, axis=2)
    
# design network
    model = Sequential()
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    #model.add(LSTM(100))
    #model.add(Dropout(0.2))
    model.add(Dense(1,init='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
#fitting the lSTM model
    model.fit(X_train,Y_train,epochs=5, batch_size=32,shuffle=True,validation_data=(X_test,Y_test))
    
    
#predicting using the model
    Y_pred = model.predict(X_test)
    Y_pred = np.array(pd.DataFrame(Y_pred)[0].apply(lambda x: 1 if x>0.5 else 0))
    Y_test_prob = model.predict_proba(X_test)
    
#AUC and F1 score
    print("For measurement of "+ values.name)
    print("AUC: ",roc_auc_score(Y_test,Y_test_prob))
    print("F1 Score",f1_score(Y_test,Y_pred.round()))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 15413 samples, validate on 6511 samples
Epoch 1/5





15413/15413 [==============================] - 337s 22ms/step - loss: 0.6890 - acc: 0.5517 - val_loss: 0.6840 - val_acc: 0.5683
Epoch 2/5
15413/15413 [==============================] - 327s 21ms/step - loss: 0.6875 - acc: 0.5609 - val_loss: 0.6837 - val_acc: 0.5683
Epoch 3/5
15413/15413 [==============================] - 327s 21ms/step - loss: 0.6874 - acc: 0.5602 - val_loss: 0.6854 - val_acc: 0.5684
Epoch 4/5
15413/15413 [==============================] - 329s 21ms/step - loss: 0.6864 - acc: 0.5619 - val_loss: 0.6851 - val_acc: 0.5686
Epoch 5/5
15413/15413 [==============================] - 330s 21ms/step - loss: 0.6858 - acc: 0.5626 - val_loss: 0.6847 - val_acc: 0.5686
For measurement of HEART_RATE
AUC:  0.520

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Train on 15405 samples, validate on 6508 samples
Epoch 1/5
15405/15405 [==============================] - 340s 22ms/step - loss: 0.6900 - acc: 0.5494 - val_loss: 0.6832 - val_acc: 0.5687
Epoch 2/5
15405/15405 [==============================] - 339s 22ms/step - loss: 0.6859 - acc: 0.5574 - val_loss: 0.6840 - val_acc: 0.5687
Epoch 3/5
15405/15405 [==============================] - 339s 22ms/step - loss: 0.6850 - acc: 0.5609 - val_loss: 0.6838 - val_acc: 0.5685
Epoch 4/5
15405/15405 [==============================] - 340s 22ms/step - loss: 0.6844 - acc: 0.5609 - val_loss: 0.6844 - val_acc: 0.5685
Epoch 5/5
15405/15405 [==============================] - 339s 22ms/step - loss: 0.6837 - acc: 0.5598 - val_loss: 0.6826 - val_acc: 0.5684
For measurement of RESPIRATORY_RATE
AUC:  0.5434834190626417
F1 Score 0.000711490572749911


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Train on 15400 samples, validate on 6504 samples
Epoch 1/5
15400/15400 [==============================] - 345s 22ms/step - loss: 0.6911 - acc: 0.5470 - val_loss: 0.6852 - val_acc: 0.5680
Epoch 2/5
15400/15400 [==============================] - 345s 22ms/step - loss: 0.6886 - acc: 0.5557 - val_loss: 0.6877 - val_acc: 0.5689
Epoch 3/5
15400/15400 [==============================] - 345s 22ms/step - loss: 0.6870 - acc: 0.5571 - val_loss: 0.6848 - val_acc: 0.5680
Epoch 4/5
15400/15400 [==============================] - 344s 22ms/step - loss: 0.6870 - acc: 0.5605 - val_loss: 0.6848 - val_acc: 0.5684
Epoch 5/5
15400/15400 [==============================] - 342s 22ms/step - loss: 0.6870 - acc: 0.5608 - val_loss: 0.6854 - val_acc: 0.5680
For measurement of O2_SATURATION
AUC:  0.5169902536059096
F1 Score 0.009167842031029619


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Train on 15234 samples, validate on 6442 samples
Epoch 1/5
15234/15234 [==============================] - 337s 22ms/step - loss: 0.6890 - acc: 0.5528 - val_loss: 0.6850 - val_acc: 0.5703
Epoch 2/5
15234/15234 [==============================] - 336s 22ms/step - loss: 0.6858 - acc: 0.5587 - val_loss: 0.6838 - val_acc: 0.5689
Epoch 3/5
15234/15234 [==============================] - 335s 22ms/step - loss: 0.6855 - acc: 0.5592 - val_loss: 0.6836 - val_acc: 0.5691
Epoch 4/5
15234/15234 [==============================] - 335s 22ms/step - loss: 0.6846 - acc: 0.5621 - val_loss: 0.6822 - val_acc: 0.5692
Epoch 5/5
15234/15234 [==============================] - 336s 22ms/step - loss: 0.6853 - acc: 0.5614 - val_loss: 0.6820 - val_acc: 0.5692
For measurement of BLOOD_PRESSURE
AUC:  0.5308644728846041
F1 Score 0.001439366678661389
